<a href="https://colab.research.google.com/github/RoyElkabetz/Text-Summarization-with-Deep-Learning/blob/main/Text_Summarization_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## uncomment only if running from google.colab
# clone the git reposetory
!git clone https://github.com/RoyElkabetz/Text-Summarization-with-Deep-Learning
# add path to .py files for import
import sys
sys.path.insert(1, "/content/Text-Summarization-with-Deep-Learning")

Cloning into 'Text-Summarization-with-Deep-Learning'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 47 (delta 25), reused 6 (delta 0), pack-reused 0
Unpacking objects: 100% (47/47), done.


In [2]:
## uncomment if you want to mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


[How to connect Telegram to Python](https://medium.com/@mycodingblog/get-telegram-notification-when-python-script-finishes-running-a54f12822cdc)

In [ ]:
## uncomment if you want to use telegram for alerts to phone
!pip install python-telegram-bot

import json
import telegram

def send_message(message):
    "This function sends a message to telegram acount"
    with open('/content/gdrive/MyDrive/telegram_bot_key.txt', 'r') as keys_file:
        k = json.load(keys_file)
        token = k['telegram_token']
        chat_id = k['telegram_chat_id']
    bot = telegram.Bot(token=token)
    bot.sendMessage(chat_id=chat_id, text=message)

send_message('Telegram pinger is connected.')


In [23]:
# imports for the practice (you can add more if you need)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import os
import math
from tqdm.autonotebook import tqdm
from IPython.display import HTML, display, clear_output

# pytorch
import torch
import torch.nn as nn
import torchtext
import torchtext.legacy.data as data
import torchtext.legacy.datasets as datasets
import torch.nn.functional as f
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import torch.optim as optim
from collections import Counter
from torchtext.vocab import Vocab

# # optuna
# pip install optuna
# import optuna

# Useful Text Preprocessing Functions

In [20]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

# Amazon-Fine-Food-Reviews Dataset
The link to dataset in Kaggle can be found [here](https://www.kaggle.com/snap/amazon-fine-food-reviews).

In [28]:
dataset_link = '/content/gdrive/MyDrive/Datasets/Text/Amazon_Fine_Food_Reviews.csv'
#data = pd.read_csv(dataset_link, usecols=['Summary', 'Text'])
data = pd.read_csv(dataset_link)
data.drop_duplicates(subset=['Text'], inplace=True) #  dropping duplicates
data.dropna(axis=0, inplace=True)                   #  dropping na
print('Number of Text - Summary samples: {}'.format(len(data)))
data.head()

Number of Text - Summary samples: 393565


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [30]:
# demo_torchtext.py
# torchtext 0.8 - undergoing massive rearchitecting

import warnings   # being significantly revamped
import random
import numpy as np
import torch as T
import torchtext.legacy as tt

device = T.device("cpu")
warnings.filterwarnings("ignore")  # else warnings spew
random.seed(2)
T.manual_seed(1)
np.random.seed(1)

print("\nBegin torchtext from raw data demo ")

print("\nCreating RAW, TEXT, TARGET Field objects ")
RAW = tt.data.RawField()
TEXT = tt.data.Field(sequential=True,
                     init_token='<sos>',  # start of sequence
                     eos_token='<eos>',   # end of sequence
                     lower=True,
                     tokenize=tt.data.utils.get_tokenizer("basic_english"),)
TARGET = tt.data.Field(sequential=True,
                       use_vocab=False,
                       is_target=True)




Begin torchtext from raw data demo 

Creating RAW, TEXT, TARGET Field objects 


In [31]:
print("\nSplitting into train, valid, test ")
(train_obj, valid_obj, test_obj) = \
  tt.data.TabularDataset(
  path='/content/gdrive/MyDrive/Datasets/Text/Amazon_Fine_Food_Reviews.csv',
  format='csv',
  fields=[('Id', RAW), 
          ('ProductId', None), 
          ('UserId', None), 
          ('ProfileName', None), 
          ('HelpfulnessNumerator', None), 
          ('HelpfulnessDenominator', None), 
          ('Score', None), 
          ('Time', None),  
          ('Summary', TARGET), 
          ('Text', TEXT)])


Splitting into train, valid, test 


ValueError: ignored

In [27]:
print("\nSplitting into train, valid, test ")
(train_obj, valid_obj, test_obj) = \
  tt.data.TabularDataset.splits(
  path='/content/gdrive/MyDrive/Datasets/Text/',
  train='Amazon_Fine_Food_Reviews.csv',
  format='csv',
  fields=[('id', RAW), ('review', TEXT),
    ('label', LABEL)])




Splitting into train, valid, test 


ValueError: ignored

In [ ]:
print("\nThe \'review\' field for item [2] is: ")
print(train_obj[2].review)

print("\nCreating vocabulary object ")
TEXT.build_vocab(train_obj)
print("The idx of \'good\' is ",
  TEXT.vocab.stoi['good'])  # 13
print("The string value of 8 is ",
  TEXT.vocab.itos[8])  # 'bad'
 
print("\nCreating a train BucketIterator ")

train_iter  = tt.data.BucketIterator(
  dataset=train_obj,
  batch_size = 2,
  sort_key=lambda x: len(x.review),
  shuffle=True,
  device=device)

print("\nIterating train data (batch_size=2) ")
for item in train_iter:
  print("\n=====\n")
  print(item.id)
  print(item.review)
  print(item.label)
  
print("\nEnd of demo ")